# 타입B 중간보스

In [1]:
import re
from tqdm.notebook import tnrange
import csv
import json
from os import walk
import os
import pandas as pd
import datetime

In [2]:
# 해당 경로의 파일리스트 추출
def get_filelist(input_repo):
    file_list = []
    for (dirpath, dirnames, filenames) in walk(input_repo):
        file_list.extend(filenames)
        break

    return file_list

In [3]:
# 해당 경로의 csv파일의 경로+파일명 리스트 추출
def get_csvlist(input_repo, file_list):
    csv_list = []
    for i in range(len(file_list)):
        filename, file_extension = os.path.splitext(f'{input_repo}/{file_list[i]}')

        if file_extension == '.csv':
            csv_list.append(filename+file_extension)

    return csv_list

In [4]:
def concat_csvfile(csv_list):
    COLUMNS = ['lineNumber', 'originalText', 'transcription']
    result = pd.concat([pd.read_csv(f) for f in csv_list])    # 라인넘버 기준 안합친파일 합치기
    sorted_result_not_flat = result.sort_values(by=['lineNumber'])    # 작업물 sort by lineNumber
    sorted_result_not_flat.to_csv(f'{output_repo}/temp.csv', index=False, encoding='utf-8-sig')
    sorted_result = pd.read_csv(f'{output_repo}/temp.csv')
    os.remove(f'{output_repo}/temp.csv')
    
    return sorted_result

In [5]:
# 전체 문장에서 태그 안만 뽑음
def save_tag(sentence):
    regex = re.compile('(?<=\<{3}).*(?=\>{3})')
    result = regex.search(sentence)
    
    return result

In [6]:
# 전체 문장으로부터 태그 제거
def remove_tag(sentence):
    result = sentence.replace('<<<', '').replace('>>>', '')
        
    return result

In [7]:
# 태그 내 문장으로부터 법률어휘 추출
def draw_word(sentence_tagremoved):
    regex = re.compile('(?<=\<=).*(?=\>)')
    result = regex.search(sentence_tagremoved)
    
    try:
        return result.group()
    except:
        return 'no word'

In [8]:
# csv형태로 저장
def save_as_csv(input_repo, output_repo, result):
    year = str(datetime.datetime.now().year)[2:]
    month = str(datetime.datetime.now().month)
    day = str(datetime.datetime.now().day)
    hour = str(datetime.datetime.now().hour)
    minute = str(datetime.datetime.now().minute)

    print(f'{output_repo}의 경로에 태그안뽑기_{year}{month}{day}_{hour}{minute}.csv의 이름으로 파일 생성이 완료되었습니다.')

    result.to_csv(f'{output_repo}/태그안뽑기_{year}{month}{day}_{hour}{minute}.csv', index=False, encoding='utf-8-sig')

In [9]:
input_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/concat'
ref_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/accum_concat/법률어휘 살리기 최종'
output_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기'

ref_filename = 'total_with_lineNumber.csv'

In [10]:
file_list = get_filelist(input_repo)
csv_list = get_csvlist(input_repo, file_list)
sorted_result = concat_csvfile(csv_list)
ref_word = pd.read_csv(f'{ref_repo}/{ref_filename}')

In [14]:
result_dataframe = pd.DataFrame(columns=['lineNumber', 'word', 'originalText', 'transcription'])
for i in tnrange(len(sorted_result)):
    word = list(ref_word[ref_word['lineNumber']==sorted_result['lineNumber'][i]]['word'])[0]
    lineNumber = sorted_result['lineNumber'][i]
    originalText = list(ref_word[ref_word['lineNumber']==sorted_result['lineNumber'][i]]['org'])[0]
    transcription = sorted_result['transcription'][i]
    result_dataframe = result_dataframe.append({'lineNumber':lineNumber, 'word':word, 'originalText':originalText, 'transcription':transcription}, ignore_index=True)

  0%|          | 0/52437 [00:00<?, ?it/s]

In [15]:
result_dataframe

,lineNumber,word,originalText,transcription
0,1,호환,"원심판결 이유에 의하면, 원심은 이 사건 행위가 공정거래법 제3조의2 제1항 제3호...","<<<직전의 재판<=원심>판결 이유에>>> 의하면, <<<직전의 재판<=원심>은 이..."
1,2,호환,비록 양 발명의 구성에 상이점이 있어도 그 기술분야에서 통상의 지식을 가진 자가 보...,비록 양 발명의 구성에 <<<서로 다른<=상이> 점이 있어도>>> 그 기술분야에서 ...
2,3,호환,"[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 호환성이 있어 의장법...","[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 <<<연결되는 데 문..."
3,4,호환,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 호환컴퓨터에 대표적으...,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 <<<연결되는 데 문...
4,5,호환,"구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...","구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ..."
...,...,...,...,...
52432,75991,종용,이에 피해자의 동생은 피고인의 뜻이 확고하다면 이혼을 하더라도 살 집이라도 마련해 ...,이에 피해자의 동생은 <<<형사소송에서 검사에의해 기소당한 사람<=피고인>의 뜻이>...
52433,75992,종용,"[2] 자동차 판매대리점주 甲이 자신의 대리점에서 카마스터(car master, 자...","[2] 자동차 판매대리점주 甲이 자신의 대리점에서 카마스터(car master, 자..."
52434,75993,종용,［2］ 간통죄의 고소인과 그 배우자인 피고소인 쌍방이 혼인관계를 지속할 의사가 없었...,［2］ 간통죄의 고소인과 그 배우자인 <<<민사소송에서 소송을 당한 사람<=피고>소...
52435,75994,종용,"원심은, 원고가 피고보조참가인(이하 ""참가인""이라 한다) 등을 사업장에서 배제하게 ...","<<<직전의 재판<=원심>은, 민사소송을 일으킨 사람<=원고>이>>> <<<민사소송..."


In [16]:
save_as_csv(input_repo, output_repo, result_dataframe)

/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기의 경로에 태그안뽑기_211110_1616.csv의 이름으로 파일 생성이 완료되었습니다.
